## Part 0
Read file, data preparation

In [27]:
import pandas as pd
import numpy as np
import csv
import os
import matplotlib as plt
%matplotlib inline

###### set path
print "set path"
TDI_FILE_PATH = ''
POS_FILE_PATH = ''
SGA_FILE_PATH = ''
# TDI_FILE_PATH = "TSDtriplet.csv"
# # POS_FILE_PATH = "data_noCT/PANCAN.postprobcutoff.perSGA.csv"
# POS_FILE_PATH = "postprobthd.p=0.05.perSGA.csv"
# SGA_FILE_PATH = "SGApair.csv"
print TDI_FILE_PATH
print POS_FILE_PATH
print SGA_FILE_PATH

set path





In [12]:
def extractTriplet(TDI_FILE_PATH):
    """
    Extract Tumor-SGA-DEG-PostProb triplet (Recommend parallel processing)
    return triplet as csv file and pandas dataframe
    """
    pass

In [ ]:
df_realData = extractTriplet(REAL_FILE_PATH)
df_exp1 = extractTriplet(EXP1_FILE_PATH)
df_exp2= extractTriplet(EXP2_FILE_PATH)
df_exp3 = extractTriplet(EXP3_FILE_PATH)

## Part 1
Compare highest posterior probability per DEG distribution between real data experiment and 3 random DEG matrix permutation experiments.

In [19]:
# Plot the distribution of postprob from real data and that from random permutated data (average value) in the same figure.
plt.figure();
plt.hist(df_realData['posterior'], bins=100, alpha=0.5, histtype='step', label='real')
data_exp1 = plt.hist(df_exp1['posterior'], bins=100, alpha=0.5, histtype='step', label='exp1') 
data_exp2 = plt.hist(df_exp2['posterior'], bins=100, alpha=0.5, histtype='step', label='exp2')
data_exp3 = plt.hist(df_exp3['posterior'], bins=100, alpha=0.5, histtype='step', label='exp3')
plt.legend(loc='upper left')
plt.title('posterior probability')

## Part 2
Compare highest posterior probability per DEG distribution for a given SGA between real data experiment, 3 random DEG matrix permutation experiments and 3 single SGA permutation experiments.

In [17]:
sgas = ['TP53', 'PIK3CA', 'PTEN', 'TTN', 'ZFHX4', 'CSMD3']
subdfs = {}
for sga in sgas:
    subdfs['sga'] = df_realData[df_realData['cause_gene_name'] == sga]

# calculate the average of rand exp histogram
data_exp1 = np.hisogram(df_exp1['posterior'], bins=100, alpha=0.5, histtype='step', label='exp1') 
data_exp2 = np.histogram(df_exp2['posterior'], bins=100, alpha=0.5, histtype='step', label='exp2')
data_exp3 = np.histogram(df_exp3['posterior'], bins=100, alpha=0.5, histtype='step', label='exp3')
left, right = data_exp1[1][:-1], data_exp1[1][1:]
X = np.array([left, right]).T.flatten()
Y = np.array([(data_exp1[0] + data_exp2[0] + data_exp3[0])/3, (data_exp1[0] + data_exp2[0] + data_exp3[0])/3]).T.flatten()

# plot histograms
plt.figure(figsize=(10,8))
plt.hist(df_realData['posterior'], bins=100, alpha=0.5, histtype='step', label='real')
plt.plot(np.append(X,1),np.append(Y, 0))
for key, subdf in subdfs:
    plt.hist(subdf['posterior'], bins=100, alpha=0.5, histtype='step', label=key)
plt.legend(loc='upper left')
plt.title('posterior probability')

## Part 3
Use 5, 10, 15 DEGs to called driver. Compare the number of drivers per tumor called between real data and random permutated data.

| DEG threshold | 5        | 10    | 15 
| :------------- |:--------| :-----|:------
| real data     | 0.0      | 0.0   |  0.0
| rand exp 1    | 0.0      | 0.0   |  0.0
| rand exp 2    | 0.0      | 0.0   |  0.0
| rand exp 3    | 0.0      | 0.0   |  0.0


In [16]:
def read_file(TRIPLET_FILE_PATH, POS_FILE_PATH):  
    """
        read the tdi file and posterior file 
        return filtered dataframe of tdi results
    """
    # df = pd.read_csv("data_noCT/TSDtriplet.csv")
    # df_posterior = pd.read_csv("data_noCT/PANCAN.postprobcutoff.perSGA.csv")

    df = pd.read_csv(TDI_FILE_PATH)
    df_posterior = pd.read_csv(POS_FILE_PATH)
    tmp = dict(zip(df_posterior['Unnamed: 0'], df_posterior.PostProbcutoff))
    df['a'] = df.cause_gene_name.map(tmp)
    # cutoff by posterior
    df_updated_cutoff = df.loc[df['posterior'] >= df['a']]
    del df_updated_cutoff['a']
    return df_updated_cutoff

def driver(df, threshold=5):
    """
        input: read_file() output
        output: csv file of driver call per tumor
    """
    grouped = df.groupby('patient_name')
    tmp = []
    for name, group in grouped:
        filtergroup = group.groupby('cause_gene_name').filter(lambda x : len(x) >= threshold)
        tmp.append(filtergroup)
    driverTriplet = pd.concat(tmp)
    driverPerTumor = pd.concat(tmp)
    del driverPerTumor['result_gene_name']
    del driverPerTumor['posterior']
    driverPerTumor = driverPerTumor.drop_duplicates()
    return (driverTriplet, driverPerTumor)

def driverCallCompare():
    """
    Result: average number drivers called across all tumors (3x4 table)
    row - number of DEGs used as the threshold, i.e. 5, 10, 15
    column - col1: real data; col2-4: random permutation experiment 1-3.
    """


In [ ]:
TRIPLET_FILE_PATH = ''
EXP1_FILE_PATH = ''
EXP2_FILE_PATH = ''
EXP3_FILE_PATH = ''
POS_FILE_PATH = POS_FILE_PATH
df = read_file(TRIPLET_FILE_PATH, POS_FILE_PATH)
df_exp1 = read_file(EXP1_FILE_PATH, POS_FILE_PATH)
df_exp2 = read_file(EXP2_FILE_PATH, POS_FILE_PATH)
df_exp3 = read_file(EXP3_FILE_PATH, POS_FILE_PATH)

# call driver(df, 5), driver(df, 10), driver(df, 10)
(driverTriplet_5, driverPerTumor_5) = driver(df, 5)
(driverTriplet_10, driverPerTumor_10) = driver(df, 10)
(driverTriplet_15, driverPerTumor_15) = driver(df, 15)

(driverTriplet_5_exp1, driverPerTumor_5_exp1) = driver(df_exp1, 5)
(driverTriplet_10_exp1, driverPerTumor_10_exp1) = driver(df_exp1, 10)
(driverTriplet_15_exp1, driverPerTumor_15_exp1) = driver(df_exp1, 15)

(driverTriplet_5_exp2, driverPerTumor_5_exp2) = driver(df_exp2, 5)
(driverTriplet_10_exp2, driverPerTumor_10_exp2) = driver(df_exp2, 10)
(driverTriplet_15_exp2, driverPerTumor_15_exp2) = driver(df_exp2, 15)

(driverTriplet_5_exp3, driverPerTumor_5_exp3) = driver(df_exp3, 5)
(driverTriplet_10_exp3, driverPerTumor_10_exp3) = driver(df_exp3, 10)
(driverTriplet_15_exp3, driverPerTumor_15_exp3) = driver(df_exp3, 15)

# count driver per tumor
print "threshold, data type: value"
print "5, real data: "driverPerTumor_5.groupby('patient_name').count().mean()[0]
print "10, real data: "driverPerTumor_10.groupby('patient_name').count().mean()[0]
print "15, real data: "driverPerTumor_15.groupby('patient_name').count().mean()[0]
print "5, exp1: "driverPerTumor_5_exp1.groupby('patient_name').count().mean()[0]
print "10, exp1: "driverPerTumor_10_exp1.groupby('patient_name').count().mean()[0]
print "15, exp1: "driverPerTumor_15_exp1.groupby('patient_name').count().mean()[0]
print "5, exp2: "driverPerTumor_5_exp2.groupby('patient_name').count().mean()[0]
print "10, exp2: "driverPerTumor_10_exp2.groupby('patient_name').count().mean()[0]
print "15, exp2: "driverPerTumor_15_exp2.groupby('patient_name').count().mean()[0]
print "5, exp3: "driverPerTumor_15_exp3.groupby('patient_name').count().mean()[0]
print "10, exp3: "driverPerTumor_10_exp3.groupby('patient_name').count().mean()[0]
print "15, exp3: "driverPerTumor_15_exp3.groupby('patient_name').count().mean()[0]

## Part 4
Use 5, 10, 15 DEGs to called driver for a given SGA. Compare the number of tumors the given SGA is called as a driver in real data, 3 random DEG permutated data and single SGA permutated data.

In [11]:
# variables
sgas = ['TP53', 'PIK3CA', 'PTEN', 'TTN', 'ZFHX4', 'CSMD3']
print "threshold, data type"
for sga in sgas:
    print "========" + sga + "=============="
    print "5, real data: "driverPerTumor_5[driverPerTumor_5.cause_gene_name == sga].groupby('patient_name').count().mean()[0]
    print "10, real data: "driverPerTumor_10[driverPerTumor_10.cause_gene_name == sga].groupby('patient_name').count().mean()[0]
    print "15, real data: "driverPerTumor_15[driverPerTumor_15.cause_gene_name == sga].groupby('patient_name').count().mean()[0]
    print "5, exp1: "driverPerTumor_5_exp1[driverPerTumor_5_exp1.cause_gene_name == sga].groupby('patient_name').count().mean()[0]
    print "10, exp1: "driverPerTumor_10_exp1[driverPerTumor_10_exp1.cause_gene_name == sga].groupby('patient_name').count().mean()[0]
    print "15, exp1: "driverPerTumor_15_exp1[driverPerTumor_15_exp1.cause_gene_name == sga].groupby('patient_name').count().mean()[0]
    print "5, exp2: "driverPerTumor_5_exp2[driverPerTumor_5_exp2.cause_gene_name == sga].groupby('patient_name').count().mean()[0]
    print "10, exp2: "driverPerTumor_10_exp2[driverPerTumor_10_exp2.cause_gene_name == sga].groupby('patient_name').count().mean()[0]
    print "15, exp2: "driverPerTumor_15_exp2[driverPerTumor_15_exp2.cause_gene_name == sga].groupby('patient_name').count().mean()[0]
    print "5, exp3: "driverPerTumor_15_exp3[driverPerTumor_5_exp3.cause_gene_name == sga].groupby('patient_name').count().mean()[0]
    print "10, exp3: "driverPerTumor_10_exp3[driverPerTumor_10_exp3.cause_gene_name == sga].groupby('patient_name').count().mean()[0]
    print "15, exp3: "driverPerTumor_15_exp3[driverPerTumor_15_exp3.cause_gene_name == sga].groupby('patient_name').count().mean()[0]    

## Part 5
Calculate average driver call rate for well known drivers collected from Kandoth and Lawrence papers. What percentage of well known drivers are called as TDI drivers in at least one tumor? (Use 5, 10, 15 DEGs to called driver.)